## **Object-Oriented :** _SOLID_

#### _Les propriétés SOLID en Orienté Objet_

🟢 `complete`

---

1. **Principe**
    * Définitions
2. **Propriétés**
    * Single Responsibility
    * Open/Closed 
    * Liskov Substitution
    * Interface Segregation
    * Dependency Inversion

`---`

⚠️ Les déclarations ci-dessous sont dépourvues des contraintes "d'interfaces formelles", de "propriétés privées", "d'encapsulation" afin de faciliter la compréhension d'une architecture _SOLID_

**Built-in**

In [31]:
from abc import ABC, abstractmethod

---
### **1.** Principe

##### **1.1** - Définitions

**S** : Responsabilité unique `Single Responsibility`

* Une classe n'a **qu'une seule responsabilité**.
* Éviter impérativement les _God Objects_.

**O** : Ouvert à l'extension / Fermé à la modification `Open/Closed`

* Une classe doit pouvoir être **étendue**. 
* Eviter de _modifier_ le code source préexistant.

**L** : Substitution de Liskov `Liskov Substitution`

* Une instance d'un type de base doit pouvoir être **substitué par un sous-type de celui-ci** sans altérer le comportement du programme.
* Eviter aux méthodes de devoir _déterminer_ si un de ses paramètres est d'un sous-type précis et de changer le comportement du programme en fonction du sous-type.

**I** : Ségrégation des interfaces `Interface segregation`

* Définir **plusieurs interfaces précises** pour faciliter leur implémentation par les classes
* Eviter les _interfaces généralisées_.

**D** : Inversion des dépendances `Dependency inversion`

* Une classe doit toujours **dépendre des abstractions** en cassant les références entres différentes classes (découplage)
* Eviter auc classes de _dépendre des implémentations_. 

---
### **2.** Propriétés

##### **2.1** - Single Responsibility

❌ **Incorrect**

In [32]:
# Classe 'God Object' où les employés sont également gestionnaires des équipes
class Employee :

    def __init__(self, firstname:str, lastname:str, teams:dict) -> None :
        self.__firstname = firstname
        self.__lastname = lastname
        self.__teams = teams

    @property
    def teams(self) :
        return self.__teams

    def create_teams(self, name:str) -> None :
        if not name in self.__teams :
            self.__teams[name] = []

    def add_member(self, name:str, employee:str) -> None :
        if (name in self.__teams) and not (employee in self.__teams[name]) :
            self.__teams[name].append(employee)
    
    def desc(self) -> str :
        return (f"{self.__firstname} {self.__lastname}")

In [33]:
# Implémentation de Margaret
margaret = Employee("Margaret", "Hamilton", {'software':[]})
margaret.add_member('software', margaret.desc())
margaret.create_teams('marketing')

margaret.teams

{'software': ['Margaret Hamilton'], 'marketing': []}

In [34]:
# Implémentation d'Alan, intègre l'équipe marketing de Margaret
alan = Employee("Alan", "Turing", margaret.teams)
alan.add_member('marketing', alan.desc())

alan.teams

{'software': ['Margaret Hamilton'], 'marketing': ['Alan Turing']}

✔️ **Correct**

In [54]:
# Classes distinctes
class Employee :

    def __init__(self, firstname:str, lastname:str) -> None :
        self.__firstname = firstname
        self.__lastname = lastname


class Team :

    def __init__(self, name:str) -> None :
        self.__name = name
        self.__members = []

    @property
    def members(self) -> list :
        return self.__members

    @members.setter
    def members(self, employee:Employee) -> None :
        if not employee in self.__members :
            self.__members.append(employee)

In [55]:
# Implémentation des équipes
software = Team("Software Engineering")
research = Team("Research & Development")

In [56]:
# Implémentation de margaret
margaret = Employee("Margaret", "Hamilton")
software.members = margaret
research.members = margaret

In [57]:
# Implémentation d'alan
alan = Employee("Alan", "Turing")
research.members = alan

In [59]:
# Résultats 
print(research.members)
print(software.members)

[<__main__.Employee object at 0x000002425D98E8C0>, <__main__.Employee object at 0x000002425D98EE00>]


##### **2.2** - Open/Closed 

❌ **Incorrect**

In [40]:
class Supplier :

    payment_type = 'invoice'
    
    def __init__(self, name:str, invoice:int) -> None :
        self.name = name
        self.invoice = invoice


class Worker :
    
    payment_type = 'salary'

    def __init__(self, fullname:str, salary:float) -> None :
        self.fullname = fullname
        self.salary = salary


# Classe complexe a maintenir, devra être modifiée si ajout d'un nouveau destinataire de paiement
class Company :

    def __init__(self, to_pay:list[Worker|Supplier]) -> None :
        self.__to_pay = to_pay

    def send_paiments(self) -> None :
        for entity in self.__to_pay :
            if entity.payment_type :
                if entity.payment_type == 'invoice' :
                    # entity is a Supplier
                    print(f"{entity.name} est payé {entity.invoice} €")
                elif entity.payment_type == 'salary' :
                    # entity is a Worker
                    print(f"{entity.fullname} est payé·e {entity.salary} €")
                else :
                    print(f"[!] - Paiement impossible : 'entity' non-reconnue...")

    def add_paiment_type(self, p) -> None :
        self.__to_pay.append(p)


# Le problème commence ici : nouvelle entité à payer, cas imprévu par la classe Company
class TaxOffice : 
    
    payment_type = "tax"

    def __init__(self, tax:float) -> None :
        self.tax = tax

In [41]:
# Implémentation et paiements : Ok jusque là
company = Company([
    Worker("Margaret Hamilton", 10_000),
    Worker("Alan Turing", 10_000),
    Supplier("MIT", 150_000)
])

company.send_paiments()

Margaret Hamilton est payé·e 10000 €
Alan Turing est payé·e 10000 €
MIT est payé 150000 €


In [42]:
# Nouvelle entité à payer : la classe 'Company' doit être modifier pour palier au nouveau mode de paiment
company.add_paiment_type(TaxOffice(21_000))

company.send_paiments()

Margaret Hamilton est payé·e 10000 €
Alan Turing est payé·e 10000 €
MIT est payé 150000 €
[!] - Paiement impossible : 'entity' non-reconnue...


✔️ **Correct**

In [43]:
# Création d'une interface qui va servir à garantir la disponibilité du mode de paiement pour chaque classe dépendante (polymorphisme)
class InterafacePayable :
    
    def get_paid(self) -> None : pass


class Provider(InterafacePayable) :

    def __init__(self, name:str, invoice:int) -> None :
        self.name = name
        self.invoice = invoice

    def get_paid(self) -> None :
        print(f"Provider {self.name} est payé {self.invoice} €")


class Operative(InterafacePayable) :
    
    def __init__(self, fullname:str, salary:float) -> None :
        self.fullname = fullname
        self.salary = salary

    def get_paid(self) -> None :
        print(f"Operative {self.fullname} est payé·e {self.salary} €")


class TaxAgency(InterafacePayable) :

    def __init__(self, tax:float) -> None :
        self.tax = tax

    def get_paid(self) -> None :
        print(f"TaxOffice est payé {self.tax} €")


class Employer :

    def send_paiments(self, to_pay:list[InterafacePayable]) -> None :
        for entity in to_pay :
            entity.get_paid()

In [44]:
# Implémentations et paiement
company = Employer()

company.send_paiments([
    Operative("Margaret Hamilton", 10_000),
    Operative("Alan Turing", 10_000),
    Provider("MIT", 150_000),
    TaxAgency(21_000)
])

Operative Margaret Hamilton est payé·e 10000 €
Operative Alan Turing est payé·e 10000 €
Provider MIT est payé 150000 €
TaxOffice est payé 21000 €


##### **2.3** - Liskov Substitution

❌ **Incorrect**

In [45]:
class AbstractEmployee(ABC) :
    
    def __init__(self, fullname:str, salary:int) -> None :
        self.fullname = fullname
        self.salary = salary

    
class Trader(AbstractEmployee) :

    def __init__(self, fullname:str, salary:int, commission:float, total_sales:int) -> None :
        super().__init__(fullname, salary)
        self.commission = commission
        self.total_sales = total_sales


class Engineer(AbstractEmployee) :
    
    def __init__(self, fullname:str, salary:int) -> None :
        super().__init__(fullname, salary)


# La méthode send_paiments doit distinguer le sous-type car il contiend une notion de salaire variable via les propriétés `commission` et `total_sales`
class Enterprise :

    def send_payments(self, employee_list:list[AbstractEmployee]) : 
        for employee in employee_list :
            if isinstance(employee, Trader) :
                # employee is a Trader
                prime = (employee.commission / 100) * employee.total_sales
                print(f"Trader {employee.fullname} est payé·e {employee.salary + prime} €")
            else : 
                # employee is an Engineer
                print(f"Engineer {employee.fullname} est payé·e {employee.salary} €")

In [46]:
# Implémentation et paiements
bill = Trader("Bill Gates", 10_000, 6.5, 242)
ada = Engineer("Ada Lovelace", 10_000)
enterprise = Enterprise()

enterprise.send_payments([bill, ada])

Trader Bill Gates est payé·e 10015.73 €
Engineer Ada Lovelace est payé·e 10000 €


✔️ **Correct**

In [47]:
class AbstractPerson(ABC) :
    
    def __init__(self, fullname:str, salary:int) -> None :
        self.fullname = fullname
        self.salary = salary

    @abstractmethod
    def compute_salary(self) -> float : pass


class Marketer(AbstractPerson) :

    def __init__(self, fullname:str, salary:int, commission:float, total_sales:int) -> None :
        super().__init__(fullname, salary)
        self.__commission = commission
        self.__total_sales = total_sales

    def compute_salary(self) -> float :
        return ((self.__commission / 100) * self.__total_sales) + self.salary


class Developer(AbstractPerson) :
    
    def __init__(self, fullname:str, salary:int) -> None :
        super().__init__(fullname, salary)

    def compute_salary(self) -> float :
        return self.salary


class Boss :

    def send_payments(self, people:list[AbstractPerson]) -> None :
        for person in people :
            print(f"'{person.__class__.__name__}' {person.fullname} est payé·e {person.compute_salary()} €")

In [48]:
# Implémentation et paiements
bill = Marketer("Bill Gates", 10_000, 6.5, 242)
ada = Developer("Ada Lovelace", 10_000)
ugo = Boss()

ugo.send_payments([bill, ada])

'Marketer' Bill Gates est payé·e 10015.73 €
'Developer' Ada Lovelace est payé·e 10000 €


##### **2.4** - Interface Segregation

❌ **Incorrect**

In [ ]:
class InterfaceEmployee :

    id:int = 0
    firstname:str = ''
    lastname:str = ''
    salary:float = 0.0


class InterfaceTeam :

    name:str = ''
    employees:list = []


# Cette interface définit plusieurs méthodes qui cibles différentes responsabilités, la classe qui implémente cette interface n'aura probablement pas besoin des instances Employee et Team
class InterfaceDirectory :

    def get_one(name:str) -> Team : pass
    
    def get_all() -> list : pass

    def add(employee:Employee) -> None : pass

    def delete(employee:Employee, team:Team) -> None : pass

    def add_to(employee:Employee, team:Team) -> None : pass


# Les méthodes qui ne correspondent pas à la responsabilité de la classe sont obligées d'être implémenté avec une Erreur
class EmployeeManager(InterfaceDirectory) :

    def __init__(self) -> None :
        self.__employees:list = []

    def get_one(self, name:str) -> Team : 
        raise NotImplementedError
    
    def get_all(self) -> list : 
        raise NotImplementedError

    def add(self, employee:Employee) -> None : 
        self.__employees.append(employee)

    def delete(self, employee:Employee, team:Team) -> None : 
        if employee in self.__employees :
            self.__employees.remove(employee)

    def add_to(self, employee:Employee, team:Team) -> None : 
        raise NotImplementedError

✔️ **Correct**

In [ ]:
# Chaque interface gère son propre buisiness et la classe EmployeeManagement n'implémente que les méthodes définies par l'ineterface dont elle dépend
class InterfaceTeamManager :
    
    def get_one(name:str) -> Team : pass
    
    def get_all() -> list : pass

    def add_to(employee:Employee, team:Team) -> None : pass


class InterfaceEmployeeManager :

    def add(employee:Employee) -> None : pass

    def delete(employee:Employee) -> None : pass


class EmployeeManagement(InterfaceEmployeeManager) :

    def __init__(self) -> None :
        self.__employees:list = []

    def add(self, employee:Employee) -> None : 
        self.__employees.append(employee)

    def delete(self, employee:Employee) -> None : 
        if employee in self.__employees :
            self.__employees.remove(employee)

##### **2.5** - Dependency Inversion

❌ **Incorrect**

In [61]:
class EmployeeDirectory :

    __employees = list()

    def add(employee:Employee) -> None :
        EmployeeDirectory.__employees.append(employee)

    def delete(employee:Employee) -> None :
        if employee in EmployeeDirectory.__employees :
            EmployeeDirectory.__employees.remove( employee )


# L'instanciation se déroule dans la classe 'CompanyLtd', la rendant strictement dépendante de la classe 'EmployeeDirectory'
class CompanyLdt :

    def __init__(self) -> None :
        self.__employee_directory = EmployeeDirectory()

In [62]:
# Instanciation et exploitation
ldt = CompanyLdt()

✔️ **Correct**

In [65]:
# Interface de référence pour l'implémentation
class InterfaceEmployeeDirectory :

    def add(employee:Employee) -> None : pass

    def delete(employee:Employee) -> None : pass


class EmployeeDirectoryImpl(InterfaceEmployeeDirectory) : 

    __employees = list()

    def add(self, employee:Employee) -> None :
        EmployeeDirectoryImpl.__employees.append(employee)

    def delete(self, employee:Employee) -> None :
        if employee in EmployeeDirectoryImpl.__employees :
            EmployeeDirectoryImpl.__employees.remove(employee)


class MyLtdCompany :

    def __init__(self, employee_directory:InterfaceEmployeeDirectory) -> None :
        self.__employee_directory = employee_directory

In [67]:
# Implémentation et exploitation
my_ldt_company = MyLtdCompany(
    EmployeeDirectoryImpl()
)

In [68]:
# Changement de classe
class New_EmployeeDirectoryImpl(InterfaceEmployeeDirectory) : 

    __employees = list()

    def _add(self, employee:Employee) -> None :
        New_EmployeeDirectoryImpl.__employees.append(employee)

    def delete(self, employee:Employee) -> None :
        if employee in New_EmployeeDirectoryImpl.__employees :
            New_EmployeeDirectoryImpl.__employees.remove(employee)

    
# l'implémentation peut être mise à jour
my_ldt_company = MyLtdCompany(
    New_EmployeeDirectoryImpl()
)